In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 22.2 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 895 kB 37.9 MB/s 
     |████████████████████████████████| 596 kB 32.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Kakao

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

In [11]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b',
    bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]')

In [12]:
tokenizer.tokenize('인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던')

['ìĿ¸ê°Ħ',
 'ì²ĺëŁ¼',
 'ĠìĥĿê°ģ',
 'íķĺê³ł',
 ',',
 'ĠíĸīëıĻ',
 'íķĺ',
 'ëĬĶ',
 "Ġ'",
 'ì§ĢëĬ¥',
 "'",
 'ìĿĦ',
 'ĠíĨµíķ´',
 'ĠìĿ¸ë¥ĺ',
 'ê°Ģ',
 'ĠìĿ´ìłľ',
 'ê¹Įì§Ģ',
 'ĠíĴĢ',
 'ì§Ģ',
 'Ġëª»',
 'íĸĪ',
 'ëįĺ']

In [23]:
'ìĿ¸ê°Ħ'.encode('utf-8')

b'\xc3\xac\xc4\xbf\xc2\xb8\xc3\xaa\xc2\xb0\xc4\xa6'

In [13]:
tokens = tokenizer.encode('인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던', return_tensors='pt') #.to(device='cuda', non_blocking=True)
tokens

tensor([[ 6577,  1290,  1032, 12519,   118,  2243,   385,   378,   882,  6261,
           113,   387,  1132,  5321,   402,  2092,   841,  2182,   404,   993,
           551,   726]])

In [16]:
decoded = tokenizer.batch_decode(tokens)
decoded

["인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던"]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt',
    revision='KoGPT6B-ryan1.5b',
    pad_token_id=tokenizer.eos_token_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True) #.to(device='cuda', non_blocking=True)
_ = model.eval()

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.0G [00:00<?, ?B/s]

In [ ]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt') #.to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)  # print: 인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제의 해답을 찾을 수 있을 것이다. 과학기술이 고도로 발달한 21세기를 살아갈 우리 아이들에게 가장 필요한 것은 사고력 훈련이다. 사고력 훈련을 통해, 세상


SKT

In [7]:
from transformers import PreTrainedTokenizerFast

In [24]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [9]:
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [25]:
tokenizer.tokenize("코로나19로 인한 사회적 거리두기")

['▁코', '로나', '19', '로', '▁인한', '▁사회적', '▁거리', '두', '기']

In [10]:
tokenizer.encode("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

[25906,
 8702,
 7801,
 25856,
 34407,
 10528,
 422,
 426,
 18258,
 9241,
 12521,
 254,
 110,
 162]

In [ ]:
import torch
from transformers import GPT2LMHeadModel


In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                         max_length=128,
                         repetition_penalty=2.0,
                         pad_token_id=tokenizer.pad_token_id,
                         eos_token_id=tokenizer.eos_token_id,
                         bos_token_id=tokenizer.bos_token_id,
                         use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)
'''
근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다...
'''